Pada sub-modul ini kita akan belajar untuk menjalankan model yang telah kita buat untuk melakukan tugas prediksi pada lingkup pemrosesan bahasa alami (Natural Language Processing) yang dijalankan pada Web Browser menggunakan TensorFlow.js. Adapun pengetahuan tambahan tentang HTML, CSS, JavaScript, atau Chrome Dev Tools dapat membantu anda memahami sub-modul ini dengan baik.

Kita akan belajar bagaimana:
*   Memuat dan menjalankan model pada Web Browser dengan TensorFlow.js API
*   Membuat prediksi dari input yang diberikanm

Untuk latihannya kita akan membuat laman web yang dapat memprediksi sentimen dari sebuah review restoran. Model yang digunakan telah dilatih menggunakan data review restoran Yelp (selengkapnya tentang pembuatan model ini ada pada Modul 6).

Proyek utuh dari latihan ini dapat Anda unduh pada [tautan](https://github.com/aflita/nlp-in-tfjs/blob/master/nlp_in_tfjs.ipynb) berikut.

## Kebutuhan Sistem:
Untuk latihan, kita akan menjalankan laman web secara lokal di komputer dengan menggunakan web server lokal. Fungsi-fungsi pada JavaScript harus dijalankan melalui HTTP request atau pemanggilan file lokal. Perangkat lunak yang dibutuhkan adalah:

*   Browser internet versi terbaru (Chrome).
*   Text editor (Sublime Text, Bracker, Notepad ++, dan lain-lain).
*   Server web yang ter-install di komputer (Web Server for Chrome).

Untuk mengaktifkan Web Server for Chrome, geser toggle hingga status WebServer:STARTED, kemudian pilih folder project. Salin http://127.0.0.1:8887 pada alamat browser lalu buka file index.html.


## Menggunakan TensorFlow.js API
Buatlah sebuah laman web sederhana dengan nama index.html. Tambahkan script di bawah untuk dapat menggunakan TensorFlow.js API:

<html>
  <head>
    <title>Tensorflow.js: Sentiment Analysis Demo</title>
    
    <!-- Load TensorFlow.js -->
    <script src = "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs/dist/tf.min.js"></script>
  </head>
  <body>
    <script src="predict.js"></script>
  </body>
</html>



Deklarasikan script untuk memanggil Tensorflow.js API pada tag `<script>` di file html kita. Tag ini dapat kita letakkan di dalam `<head>` atau `<body>` file html.

## Menyiapkan Model dan Metadata
Setelah mempelajari bagaimana mengonversi model agar kompatibel dengan Tensorflow.js, implementasikan model tersebut untuk melakukan prediksi. Buat file JavaScript predict.js dan tambahkan kode berikut untuk memuat model dan metadata:

let model;
let word2index;
 
async function init(){
    model = await tf.loadLayersModel('http://127.0.0.1:8887/model.json');
    isModelLoaded = true;
 
    const word2indexjson = await fetch('http://127.0.0.1:8887/word2index.json');
    word2index = await word2indexjson.json();
 
    console.log(model.summary());
    console.log('Model & Metadata Loaded Succesfully');
}



---



In [4]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('/content/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df.shape

(1000, 2)

In [5]:
df.head

<bound method NDFrame.head of                                               sentence  label
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]>

In [6]:
# convert to lowercase
df['sentence'] = df['sentence'].str.lower()

In [15]:
# remove stopwords

#from nltk.corpus import stopwords #comment jika Error dan gunakan 2 sintaks dibawah
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
stop = set(stopwords.words('english'))
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,sentence,label
0,wow... loved place.,1
1,crust good.,0
2,tasty texture nasty.,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices.,1


In [17]:
vocab_size = 2000
oov_tok = "<OOV>"
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' #remove symbols

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok, filters = filt)
tokenizer.fit_on_texts(df['sentence'].values)

word2index = tokenizer.word_index
print(len(word2index))

1998




---



Metadata adalah file berisi dictionary word index yang didapatkan dari hasil tokenisasi di TensorFlow. Untuk mendapatkan file metadata, jalankan kode berikut pada Notebook Anda:

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
 
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(df['sentence'].values)
 
word2index = tokenizer.word_index

Setelah word index didapatkan, ubah ke dalam bentuk file JSON kemudian serialisasi variabel tersebut agar dapat diunduh ke komputer lokal kita menggunakan sintaks dump:

In [19]:
import json
with open('word2index.json', 'w') as fp:
    json.dump(word2index, fp)

Untuk melihat metadata word index yang digunakan pada latihan ini, buka [JSON Viewer](https://codebeautify.org/jsonviewer). Pilih ‘Load URL’ dan salin [URL](https://raw.githubusercontent.com/aflita/nlp-in-tfjs/master/word2index.json) ini.

Jika model berhasil dimuat, kode di atas akan menampilkan model summary pada Console. Untuk melihat Console pada Chrome, tekan ctrl + shift + J, lalu pilih tab Console.

Jika model sudah berhasil dimuat, maka model sudah dapat digunakan untuk memprediksi dengan memanggil sintaks model.`predict()` dan sedikit adaptasi untuk inputan yang diberikan. Perhatikan baik-baik model summary pada screenshot di atas. Model menerima input dalam bentuk [null, 18]. Bentuk null di sini artinya nilai ini fleksibel dan dapat berubah-ubah sesuai input yang dimasukkan dan dapat berupa jumlah data pada suatu batch. 18 adalah panjang maksimum kalimat dalam bentuk padded sequence. Padded sequence ini artinya suatu kalimat sudah kita ubah menjadi bentuk angka-angka sesuai dari word index, kemudian kita tambahkan padding sampai panjang maksimum yaitu 18. 

Tips: Selalu gunakan fungsi async dan await ketika memuat model atau data berukuran besar karena kita harus menunggu proses ini sampai mengembalikan hasil. Jika tidak menggunakan fungsi asynchronous, fungsi bisa memblokir proses utama dan memperburuk user experience mengingat JavaScript menggunakan pemrosesan single-threaded.



---



In [21]:
max_length =  max(len(values.split()) for i, values in enumerate(df['sentence']))
max_length

18

In [22]:
trunc_type='post'

all_seq = tokenizer.texts_to_sequences(df['sentence'].values)
all_padded = pad_sequences(all_seq, maxlen = max_length, padding = trunc_type)
all_padded.shape

(1000, 18)

In [26]:
# split train and test sets
from sklearn.model_selection import train_test_split

X = all_padded
#y = pd.get_dummies(df['label'].values)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

#kalimat = df['sentence'].values
#y = df['label'].values

#kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, 
#                                                                test_size=0.2, random_state=1000)

(800, 18) (800,)
(200, 18) (200,)


In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim= vocab_size, output_dim=16, input_length= max_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 16)            32000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 54,321
Trainable params: 54,321
Non-trainable params: 0
_________________________________________________________________


In [28]:
num_epochs = 30
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/30
25/25 [==============================] - 8s 31ms/step - loss: 0.6935 - accuracy: 0.5063 - val_loss: 0.6937 - val_accuracy: 0.4800
Epoch 2/30
25/25 [==============================] - 0s 9ms/step - loss: 0.6933 - accuracy: 0.5050 - val_loss: 0.6932 - val_accuracy: 0.4800
Epoch 3/30
25/25 [==============================] - 0s 9ms/step - loss: 0.6656 - accuracy: 0.5738 - val_loss: 0.7018 - val_accuracy: 0.6150
Epoch 4/30
25/25 [==============================] - 0s 9ms/step - loss: 0.3188 - accuracy: 0.9013 - val_loss: 0.6832 - val_accuracy: 0.6800
Epoch 5/30
25/25 [==============================] - 0s 9ms/step - loss: 0.1440 - accuracy: 0.9588 - val_loss: 0.7916 - val_accuracy: 0.7150
Epoch 6/30
25/25 [==============================] - 0s 8ms/step - loss: 0.0486 - accuracy: 0.9850 - val_loss: 0.8633 - val_accuracy: 0.7050
Epoch 7/30
25/25 [==============================] - 0s 9ms/step - loss: 0.0222 - accuracy: 0.9962 - val_loss: 1.1984 - val_accuracy: 0.7250
Epoch 8/30
25/25 [=

In [29]:
#def toSequence(sentence):
#  pad = []
#  for stc in sentence.split():
#    if stc.lower() in word2index.keys(): 
#      pad.append(word2index[stc.lower()])
#    else: 
#      continue
#  return pad

#pad = toSequence('affordable price and nice dessert')
#pad = [269, 353, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0 ,0,0,0,0]
#len(pad)
#model.predict([pad])

In [30]:
!pip install tensorflowjs

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [31]:
saved_model_path = '/content/mymodel/'
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: /content/mymodel/assets


INFO:tensorflow:Assets written to: /content/mymodel/assets


In [32]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  /content/mymodel/ \
  /content/modeltfjs

2021-09-28 02:36:20.710065: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-28 02:36:20.725770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-28 02:36:20.726569: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-28 02:36:20.727721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-28 02:36:20.728564: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S



---



## Membuat Prediksi

Untuk membuat prediksi dari model yang telah kita buat, dibutuhkan sedikit adaptasi untuk inputan yang diberikan. Prosesnya seperti ini:




1. Sistem menerima input berupa teks. Teks tersebut dibentuk menjadi lowercase dan split setiap kata. Setiap kata nantinya dirubah menggunakan fungsi map() untuk memetakan tiap-tiap kata menjadi sekuens berupa angka berdasarkan acuan metadata (word index).

2. Setelah memiliki acuan metadata, ubah kalimat atau teks input menjadi sekuens angka

In [33]:
pad = toSequence('affordable price and nice dessert')
pad

NameError: ignored

3. Setelah didapat sekuens berupa list angka, kita tambahkan padding dengan nilai 0 sampai sekuens mencapai panjang maksimum input yang terdefinisi yakni 18. Berikut bentuk inputan yang sudah siap dimasukkan kedalam model:

In [24]:
PaddedSequence = [1697,169,254,20,216,0,0,0,0,0,0,0,0,0,0,0,0,0]

4. Input di atas sudah dapat dimasukkan ke dalam sintaks: 

In [25]:
model.predict([paddedSequence]) 

NameError: ignored